<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Image annotation UI

Open-source annotation software for object detection and image segmentation exist, however for image classification we were not able to find a good tool. Hence this notebook provides a simple UI to label images. Each image can be annotated as one or multiple labels, or marked as "Exclude" to indicate that the image should be skipped.

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
sys.path.append("../")
import numpy as np
from utils_ic.anno_utils import AnnotationWidget

Set some parameters and show the UI. All annotations are saved to (and loaded from) a pandas dataframe in the `IM_DIR` directory with filename `ANNO_FILENAME`. 

In [3]:
LABELS = ["bird", "can", "car"]
IM_DIR = "../data/fridgeObjects/can"
ANNO_FILENAME = "cvbp_ic_annotation.csv"
IM_FILENAMES = None #Set to None to annotate all images in IM_DIR

Start the UI. Unset the "allow multi-class labeling" checkbox to enforce that each image can only be annotated with exactly one label. When in doubt what the annotation for an image should be, or for any other reason (e.g. blurry or over-exposed image), mark an image as "EXCLUDE".

<center>
<img src="anno_ui.jpg" style="width: 400px;"/>
<i>Annotation UI example</i>
</center>

In [4]:
w_anno_ui = AnnotationWidget(
    labels = LABELS,
    im_dir = IM_DIR,
    anno_path = ANNO_FILENAME, #os.path.join(IM_DIR, ANNO_FILENAME),
    im_filenames = IM_FILENAMES
)

display(w_anno_ui.show())

Loading existing annotation from cvbp_ic_annotation.csv.


Fast.ai does not support the "EXCLUDE" flag, and hence we provide an example below which loads the annotations generated using the AnnotationWidget and returns a dataframe which does not contain the excluded images. This dataframe can then be used in the typical way with fast.ai's `ImageList.from_df()` function to create an ImageList.

In [5]:
import pandas as pd

# Load annotation and only keep the non-excluded images
df_annotations = pd.read_pickle(ANNO_FILENAME) 
indices = [key for key in annotations if not annotations[key].exclude]
df_annotations_not_excluded = annotations[indices]

NameError: name 'annotations' is not defined